# Finding and Optimizing Cycle Representatives

Hey everyone!

Persistent homology (PH) is one of the canonical TDA tools that can be used to better understand data. Typically, the PH process provides a macro-scale overview of the *global* topological structure of a dataset—the structure and lifetime of holes in the dataset—and does not contain *geometric* information about important topological features—where those holes are actually located. It is natural, however, to want to understand geometric properties of the topological holes, and this geometry can contain important real-world insights when contextualized in the underlying data.

*Cycle representatives* seem like the solution to this problem. A cycle representative is a chain of simplices that surrounds a topological hole. Our ability to draw meaningful conclusions from cycle representatives suffers from a uniqueness problem; *any* chain of simplices that surrounds a hole is a cycle representative for that hole. Therefore, in big datasets a single hole can be represented by multiple cycle representatives that are potentially very different from each other.

An optimal cycle representative is a unique[^1] cycle representative for a hole that minimizes some $\operatorname{loss}$ function of the cycle. These optimal cycle representatives are useful, since they often hold some real-world meaning in the underlying dataset.

This notebook gives a brief intro to cycle optimization with code examples of how it can be implemented. The code can be (somewhat) complex, but all the code in this notebook should be functional so you don't need to perfectly understand the code to understand the cycle optimization concepts in the notebook. Some blocks can take longer (up to two minutes or so) to run. This can be sped up by running the code on a computer (instead of in Google Colab) or using a third-party optimizer (like Gurobi) which requires a license.

[^1]: Usually. Some simplicial complexes, especially ones that have a discrete filtration, will not have unique cycle representatives.

# Preliminaries
This notebook was created to be run in Python. We'll use the following packages throughout:
- **Matplotlib.** Plotting library. You could also use plotly (like OAT does), but I have rendering issues occasionally with that.
- **Numpy.** Useful numerical and array operations.
- **Scipy.** Sparse matrix implementation.
- **Pandas.** Dataframe operations. Most OAT results are dataframes which we can modify using pandas.
- **Open Applied Topology (OAT).** Persistant homology calculations. Very transparent, which is useful for much of what we'll do in this notebook
- **PuLP.** Frontend linear programming library. PuLP allows us to choose the backend we use to optimize. We'll use COIN, which I believe is installed by deafult. If it's not, see the installation instructions [here](https://github.com/coin-or/Clp).

These packages can all be installed using pip and/or conda.

In [ ]:
# install the needed extra packages
!python -m pip install oat_python pulp  # python packages
!sudo apt-get install coinor-cbc  # linear programming libraries

  Obtaining dependency information for pulp from https://files.pythonhosted.org/packages/84/45/2bb878df73b5545405faff0b0b30f72929222356387a41b50ca268951d5d/pulp-3.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/4b/fa/a7e5b95afd80d24313307f03624acc65801846fa75599034f8ceb9e2cbf6/scipy-1.15.3-cp311-cp311-macosx_14_0_arm64.whl.metadata
  Using cached scipy-1.15.3-cp311-cp311-macosx_14_0_arm64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 4.0 MB/s eta 0:00:0000:0100:01
Using cached scipy-1.15.3-cp311-cp311-macosx_14_0_arm64.whl (22.4 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=

In [ ]:
# plotting libraries
from matplotlib.collections import LineCollection, PatchCollection
from matplotlib.patches import Annulus, Polygon
from matplotlib.text import Text
import matplotlib.pyplot as plt

# standard numerical libraries
from scipy import sparse
import pandas as pd
import numpy as np

# persistant homology computations
import oat_python as oat

# linear programming
import pulp
lpsolver = pulp.COIN  # <- Slower, but installed with pulp by default
# lpsolver = pulp.GUROBI  # <- Faster, but requires the gurobipy library and a gurobi license (free for acedemics)

# Background: Simplicial Complexes, Cycles, and Persistent Homology

We'll start with some background. Much of this should feel familiar, but there are a couple topics that will be of particular importance for cycle optimization that will be emphasized here.

**Simplicial Complexes.** A *simplicical complex* is a structured set of simplices. A *simplex* is the convex hull of its vertices, so a simplex can be written as a collection of vertices $(v_1, v_2, \dots, v_{n+1})$. A simplex's dimension is the number of vertices minus 1; a 0-dimensional simplex is a point, a 1-dimensional simplex is an edge, a 2-dimensional simplex is a triangle, and so on. In this notebook, we'll consider simplicical complexes with 0-, 1-, and 2- dimensional simplices, though the methods do extend to higher dimensions[^2].

<div>
    <img src="https://drive.google.com/uc?export=view&id=1BQdVwvNdHf9J2goQpQZ7khYnJiyhkbsO" width="400"/>
    <figcaption>Example simplicial complex.</figcaption>
</div>

**Boundary Operator.** An $n$-dimensional simplex is surrounded by $(n-1)$-dimensional simplices that make up its *boundary*. Formally, we define the $n$-dimensional *boundary operator* $\partial_n$ so that
$$
    \partial_n (v_1, v_2, \dots, v_{n+1}) = \sum_{i = 1}^{n+1} (-1)^i (v_1, \dots, \hat{v}_i, \dots, v_{n+1})
$$
where the hat in $\hat{v}_i$ means $v_i$ is excluded. The sign of a simplex determines its orientation: $-(b, a) = (a, b)$.[^3]

| Simplex | Boundary | Math | Image |
| --- | --- | --- | --- |
| 0-Simplex (Point) | 0 (Nothing) | $\partial_0 (a) = 0$ | <div> <img src="https://drive.google.com/uc?export=view&id=1C-1f-d08N__iOipDOsLTDfOt7MEcha_a" width="200"/> </div> |
| 1-Simplex (Line) | 0-Simplices (Points) | $\partial_1 (ab) = a - b$ | <div> <img src="https://drive.google.com/uc?export=view&id=1PdeUHkedQ-gKP4Xqswd_Al-4hPOCJtia" width="200"/> </div> |
| 2-Simplex (Triangle) | 1-Simplices (Lines) | $\partial_1 (abc) = -bc + ac - ab$ | <div> <img src="https://drive.google.com/uc?export=view&id=1gppjG7lmjNrEqu1cHv3rmL9Uxdco9kUc" width="200"/> </div> |

The boundary operator is linear, so the boundary of a chain (linear combination) of simplices is the boundary of each individual simplex. This means

\begin{align*}
    \partial_n (-\sigma) &= - \partial_n \sigma \\
    \partial_n (\sigma_1 + \sigma_2) &= \partial_n \sigma_1 + \partial_n \sigma_2.
\end{align*}

| Chain | Boundary | Math | Image |
| --- | --- | --- | --- |
| Linked Lines | Points | $\partial_0 (ab + bc + cd) = a - d$ | <div> <img src="https://drive.google.com/uc?export=view&id=1fbqezKYWBYxQuh3pNjzKMQTSo5FYxRgL" width="200"/> </div> |
| A Triangle's Boundary | 0 (Nothing) | $\partial_1 (ab + bc + ca) = a - b$ | <div> <img src="https://drive.google.com/uc?export=view&id=1dnMwBgdEYlGKVnZG2pPgxR3LG8tU4NXQ" width="200"/> </div> |
| A Cycle with Extra Bits | Points | $\partial_1 (ab + bc + ca + bd + ce) = b - d + c - e$ | <div> <img src="https://drive.google.com/uc?export=view&id=1lRFy4UXV-QJbrnD61dFSSNzuGZowIqby" width="200"/> </div> |

**Cycles.** A *cycle* in a simplicial complex is a chain of simplices that has no boundary, i.e.
$$
    \partial_n (\text{Cycle}) = 0.
$$
Since boundaries are always cycles, of particular interest are the cycles that are not a boundary of any higher dimensional simplices, since these are holes around some empty region in the simplicial complex. These form the homology group
$$
    H_n = \underbrace{\ker \partial_n}_\text{Cycles} / \underbrace{\operatorname{im} \partial_{n+1}}_\text{that aren't boundaries}.
$$
Since $\partial_n$ is a linear operator, one cycle can be represented in many ways. Any boundaries or other cycles can be added to the initial cycle to form another representative of the same hole.

<div>
    <img src="https://drive.google.com/uc?export=view&id=1bPTN0VKceanIswiiR3usFFNGDNwnQUIx" width="900"/>
    <figcaption>Example representatives for the sample hole. All three cycles surround the center gap, though they are all slightly different. The first one (Blue) is the tighest representative for the hole, the second (orange) is blue plus a boundary, and the third (green) is blue plus a cycle.</figcaption>
</div>

The cycle optimization procedures we'll implement all rely on this concept. To find an optimal cycle representative, we'll add linear combinations of cycles and boundaries together to minimize some $\operatorname{loss}$ function of the cycle.

**Filtration.** For TDA applications, we're interested in how cycles found via persistent homology in a *filtered simplicial complex* can be optimized, not just cycles in a single, static simplicial complex. A filtered simplicial complex builds the final simplicical complex by progressively adding simplices over a filtration instead of all at once.

<div>
    <img src="https://drive.google.com/uc?export=view&id=1ToBJJ-6hTcZa-CrG0G9kds_5O7GF77Vp" width="900"/>
    <figcaption>A filtered simplicial complex.</figcaption>
</div>

Cycles don't necessarily exist through the whole filtration; they have
- a birth filtration where they show up.
- a death filtration where they fully close (become a boundary).

Today, we'll be finding a cycle representative at the birth filtration, since this cycle representative will surround all iterations of the cycle. When we optimize, we'll put particular care into ensuring the cycle also is filled (dies) at the same filtration value as our initial representative.

[^2]: More on that later!
[^3]: The sign can be excluded when working in $\mathbb{Z}_2$ coefficients, but we'll need it for optimization.

# Test Case
Throughout this notebook, we'll optimize a cycle from a dataset sampled from an annulus. We'll try to find a representative for the hole in the center of the data.

In [ ]:
# config
inner_radius = 1
outer_radius = 1.5
num_points = 150
np.random.seed(100)

# generate points
r = np.random.uniform(inner_radius**2, outer_radius**2, num_points)**(1/2)  # sample points with uniform prob from an annulus
theta = np.random.uniform(0, 2 * np.pi, num_points)
pts = np.array((
        r * np.cos(theta),  # x vales
        r * np.sin(theta),  # y vales
    )).T

# plot it
fig, ax = plt.subplots(figsize=(5, 5))
ax.axis('off')  # remove frame and lables
ax.axis('equal')  # equal scaleing
ax.add_patch(Annulus((0, 0), outer_radius, outer_radius - inner_radius, color='gray', alpha=0.25))  # underlying space
ax.scatter(pts[:, 0], pts[:, 1], c='k', s=10)  # points
fig.tight_layout()

# Persistent Homology
Compute persistent homology on our point set as a Vietoris-Rips complex using the OAT pipeline. This process gives us two important objects:
1. `factored`: This will be incredibly important, since it represents the U-match factorization of the simplicial complex. This object is how OAT gives us access to the boundary matrix and allows us to compute persistent homology.
2. `homology`: This contains the information encoded in a barcode, the birth and death simplices for each feature, and a (non-optimal) cycle representative and bounding chain for the feature. We'll take a look at these in a second!

In [ ]:
## oat peristent homology pipeline
max_filtration = 2.01  # max encolosing distance
# this ^^ could be computed using oat.dissimilarity.enclosing_from_cloud(pts), but I'm going to
# use some prior knowledge about the points to choose something smaller that still works
distance_matrix = oat.dissimilarity.matrix_from_cloud(pts, max_filtration) # distance matrix
factored = oat.rust.FactoredBoundaryMatrixVr(distance_matrix, 2)  # umatch factorization object up to dimension 2
homology = factored.homology(True, True)  # compute homology with both cycle and bounding chain representatives

## plot the barcode
fig = oat.plot.barcode(homology)
fig.update_layout(width=800, height=400, margin=dict(l=20, r=20, t=20, b=20))
fig.show()

In the barcode, we see one very long-lived 1-dimensional feature, which is the one we'll be interested in.

In [ ]:
# identify the longest lifetime feature
idx = (
        homology[homology['dimension'] == 1]  # filter to just dimension 1 features
            .apply(lambda r: r['death'] - r['birth'], axis=1)  # compute the lifetime
            .idxmax()  # index (in the dataframe) of the longest lived feature
    )

idx

# An Initial Cycle Representative
If we just want *some* cycle representative, OAT makes it easy. In the homology dataframe, there is a cycle representative we can use.

OAT also gives us a (again, possibly non-optimal) *bounding chain*. Since we have a filtered simplicial complex, eventually the cycle can fill in and become a boundary. The bounding chain is the region that is within the simplicial complex at the cycle's death filtration and has the cycle representative as its boundary.

In [ ]:
# get the cycle representative and bounding chain
initial_cycle = homology.loc[idx, 'cycle representative']
initial_bounding_chain = homology.loc[idx, 'bounding chain']

# make a plot
fig, ax = plt.subplots(figsize=(5, 5))
ax.axis('off')  # remove frame and lables
ax.axis('equal')  # equal scaleing

# plot the point
ax.scatter(pts[:, 0], pts[:, 1], c='k', s=10, zorder=2)  # points

# plot the cycle
lines = LineCollection(  # we could loop and plot each line individually, but that takes longer to render. linecollections will be fast
            np.array((pts[initial_cycle['simplex'].str[0]], pts[initial_cycle['simplex'].str[1]])).swapaxes(0, 1),  # points along each ot the lines
            color='k',
            lw=1.5,
            zorder=3
        )
ax.add_collection(lines)

# plot the bounding chain
tris = PatchCollection(  # again, we could loop and use fill but thats slower than a PatchCollection
        [Polygon(pts[s]) for s in initial_bounding_chain['simplex']],
        color='gray',
        edgecolor=None,  # no edges
        alpha=0.25,
        zorder=1
    )
ax.add_collection(tris)

# some final formatting
fig.tight_layout()

We have a cycle representative that surrounds the center hole! However, there are a couple of things you may notice in this image that seem non-ideal.

First, the cycle zigs and zags around a lot, especially in the center left and bottom right. It is easy to believe that there could be a cycle that more tightly surrounds the cycle.

Second, the bounding chain overlaps itself a lot. Although its boundary is the cycle,[^4] it goes in and out of the area bounded by the cycle quite a bit and overlaps itself in those areas. Could there be a better bounding chain that avoids this?

[^4]: For now, I promise this is true. Later, we'll have a way to check for sure.

# OAT Optimization
OAT makes it easy to find an optimal cycle representative. Using the `factored` object, we can call `factored.optimize_cycle` on the birth simplex of the cycle we want an optimal cycle representative of to get a tighter representative for the cycle

In [ ]:
# optimize the cycle
birth_simplex = homology.loc[idx, 'birth simplex']
oat_optimal_cycle = factored.optimize_cycle(
        birth_simplex,
        problem_type='preserve PH basis',  # generally this is the problem youll want to use. Try help(factored.optimize_cycle) for other options
    ).loc['optimal cycle', 'chain']  # optimize_cycle returns a dataframe with a lot in it, we'll only use the oat_optimal_cycle

# make a plot
fig, ax = plt.subplots(figsize=(5, 5))
ax.axis('off')  # remove frame and lables
ax.axis('equal')  # equal scaleing

# plot the point
ax.scatter(pts[:, 0], pts[:, 1], c='k', s=10, zorder=2)  # points

# plot the initial cycle
lines = LineCollection(
            np.array((pts[initial_cycle['simplex'].str[0]], pts[initial_cycle['simplex'].str[1]])).swapaxes(0, 1),
            color='tab:orange',
            lw=1.5,
            zorder=3,
            label='Initial Cycle',
        )
ax.add_collection(lines)

# plot the optmial cycle
lines = LineCollection(
            np.array((pts[oat_optimal_cycle['simplex'].str[0]], pts[oat_optimal_cycle['simplex'].str[1]])).swapaxes(0, 1),
            color='tab:blue',
            lw=1.5,
            zorder=3,
            label='Optimal Cycle',
        )
ax.add_collection(lines)

# some final formatting
ax.legend(frameon=False, loc='upper left', fontsize='small')
fig.tight_layout()

This feels like a much better cycle representative. It's smaller and sit closer to the hole as it goes around it. If you just want a reasonable cycle representative for the hole, this is a great option.

Still, it can be useful to get a better understanding of what OAT actually is doing when it optimizes this cycle. Additionally, a handful of choices went into making this cycle representative that will become more explicit later in the notebook that you may not want in your application. To address both of these, we're going to use the objects from OAT to optimize a cycle ourselves.

# The Cycle Optimization Problem
We find optimal cycles using the fact that we can add cycles and boundaries to the initial cycle representative to get new cycle representatives. Specifically, we'll minimize some loss function of the optimal cycle representative while making sure that it is the sum of some the initial cycle representative plus other valid cycle representatives and boundaries.

A valid boundary is one that exists at the birth filtration value. This ensures that the cycle still has boundary zero at the birth filtration, making it a cycle. Valid cycles similarly need to exist at the birth filtration *and* have the additional constraint that they need to die before the cycles death filtration. This ensures that the optimal cycle representative dies at the same filtration as the initial cycle representative. If we added later dying cycles, the cycle would not close at the death filtration and there would still be some open area within it.

Formally, we can find the optimal cycle representative $\mathbf{x}$ by solving

\begin{align*}
    \underset{\mathbf{x}, \mathbf{p}, \mathbf{q}}{\operatorname{minimize}} \quad & \operatorname{loss}(\mathbf{x}) \\
    \operatorname{subject~to} \quad & \mathbf{x}[\mathcal{R}] = \mathbf{x}^{Init}[\mathcal{R}] + \partial_{n+1} [\mathcal{R}, \mathcal{P}] \mathbf{p} + A [\mathcal{R}, \mathcal{q}] \mathbf{q}
\end{align*}

where $\mathbf{x}^{Init}$ is the initial cycle representative, $\partial_{n+1}$ is the boundary operator ($n$ is the dimension of $\mathbf{x}^{Init}$), and $A$ is a basis of all cycles. The solution $\mathbf{x}$ will be a vector representation of the cycle where each entry indexes a simplex and tells us whether it should be excluded (0), included (1), or included with reversed orientation (-1). The index sets ensure that the included cycles and boundaries are valid, so

\begin{align*}
    \mathcal{P} &= \{i: \operatorname{Birth}(A[:, i]) \leq \operatorname{Birth}(\mathbf{x}^{Init}),  \operatorname{Death}(A[:, i]) \leq \operatorname{Death}(\mathbf{x}^{Orig}) ,~A[:, i] \neq \mathbf{x}^{Orig}\} \\
    \mathcal{Q} &= \{\tau \in S_{n+1} (K): \operatorname{Birth}(\tau) \leq \operatorname{Birth}(\mathbf{x}^{Init})\} \\
    \mathcal{R} &= \{\sigma \in S_n (K): \operatorname{Birth}(\sigma) \leq \operatorname{Birth}(\mathbf{x}^{Init})\}.
\end{align*}


# Building the Constraint Matrices
To solve this LP, we'll need to create the matrices $A$ and $\partial_{n+1}$. OAT gives us access to everything we need to do that!

**Boundary Matrix.** We'll start by finding $\partial_{n+1}$. From the factored object in OAT we can get the boundary matrix for all dimensions $\partial$ where
$$
    \partial = \begin{pmatrix}
        0 & \partial_1 & 0 & 0 \\
        0 & 0 & \partial_2 & 0 & \dots \\
        0 & 0 & 0 & \partial_3 \\
        0 & 0 & 0 & 0 \\
        & \vdots & & & \ddots \\
    \end{pmatrix}.
$$
Therefore, we just need to select the correct block of $\partial$ to get the part we're interested in.

In [ ]:
# indicies of the boundary matrix (what we use to slice it)
boundary_matrix_indices = factored.indices_boundary_matrix()  # list of simplices in the simplicial complex
boundary_matrix_indices['simplex'] = boundary_matrix_indices['simplex'].apply(tuple)  # make this hashable so we can merge on it (useful later)
boundary_matrix_indices['dimension'] = boundary_matrix_indices['simplex'].str.len() - 1  # make filtering on dimension faster

# the boundary matrix
boundary_matrix = boundary_matrix = (
        factored.boundary_matrix()
            .astype(float)  # oat uses fractions, which has compuational advantages within oat but are hard to work with outside of it
            [(boundary_matrix_indices['dimension'] == 1).values]  # rows for 1d simplices
            [:, (boundary_matrix_indices['dimension'] == 2).values]  # columns for 2d simplices
            # rows and columns need to be seprate bc reasons (scipy throws and error if you do them at the same time)
    )

# keep the indices important for us
dim_1_indices = boundary_matrix_indices.loc[boundary_matrix_indices['dimension'] == 1, ['simplex', 'filtration']].reset_index(drop=True)
dim_2_indices = boundary_matrix_indices.loc[boundary_matrix_indices['dimension'] == 2, ['simplex', 'filtration']].reset_index(drop=True)
del boundary_matrix_indices

# plot boundary matrix (pixel where nonzero)
plt.spy(boundary_matrix, marker=',', color='k')

**Cycle Basis.** Next, we'll create the cycle basis. To make this, we'll use the cycle representatives that we have in the homology dataframe. To vectorize them so that they are consistent with the boundary matrix, we just use `dim_1_indices` which tells us which simplices represent which row of the boundary matrix.

In [ ]:
# vectorize a cycle
def cycle_vector_indices(cycle, dim_1_indices=dim_1_indices):
    # merge onto dim_1_indices
    cycle = dim_1_indices[['simplex']].merge(
            cycle.assign(simplex=lambda x: x['simplex'].apply(tuple)),  # tuples are mergeable
            on='simplex',
            how='left',  # keep the index (numbers in index correspond to the rwo)
        ).dropna()
    cycle['coefficient'] = cycle['coefficient'].astype(float)  # oat uses a fraction datatype that wont work with other things
    cycle = cycle.reset_index(names='row')[['coefficient', 'row']]  # keep the coeffiecint and row in the matrix (can make a sparse matrix from that)

    return cycle

# vector represetnation of all cycles
cycle_basis_data = pd.concat([
        cycle_vector_indices(cyc).assign(col=i)  # each 'col' column is a unique feature
            for i, cyc in enumerate(homology[homology['dimension'] == 1]['cycle representative'])
    ])
cycle_basis = sparse.csr_array(
        (cycle_basis_data['coefficient'], (cycle_basis_data['row'], cycle_basis_data['col'])),
        shape=(len(dim_1_indices), (homology['dimension'] == 1).sum()),  # has # of dim 1 simplices rows and # of dim 1 cycles columns
    )
del cycle_basis_data

# plot cycle basis (pixel where nonzero)
plt.spy(cycle_basis, marker=',', color='k')

# Edge-Loss Optimization
To find optimal cycle representatives, OAT solves an edge-loss problem. That means it makes the cycle itself (the perimeter the bounds the hole) as small as possible.

We can implement this ourselves using $\operatorname{loss}(\mathbf{x}) = c^\top |\mathbf{x}|$ where $c$ is a vector of filtration values (distances) where the simplices show up. This is a linear program, so it's easy to solve using a number of optimization libraries. We'll use PuLP, since it's open source (i.e. free) and fast.

In [ ]:
# index sets
birth = homology.loc[idx, 'birth']
death = homology.loc[idx, 'death']
potential_edges = np.where(dim_1_indices['filtration'] <= birth)[0]  # curlR, allowed edges
potential_triangles = np.where(dim_2_indices['filtration'] <= birth)[0]  # curlP, allowed triangles
cycle_mask = (homology[homology['dimension'] == 1][['birth', 'death']] <= np.array([birth, death])).all(axis=1)
potential_cycles = np.where(cycle_mask & (cycle_mask.index != idx))[0]  # curlQ, allowed cycles
init_cycle = cycle_basis[potential_edges, np.where(cycle_mask.index == idx)[0][0]].toarray()  # xinit, initial cycle

# build the solver
problem = pulp.LpProblem('Edge-Loss_Optimization', pulp.LpMinimize)

# variables we fit
cycle_pos = pulp.LpVariable.dicts('xpos', potential_edges, lowBound=0, cat='Continuous')  # cant do absolute value, instead x = x+ - x- and both are
cycle_neg = pulp.LpVariable.dicts('xneg', potential_edges, lowBound=0, cat='Continuous')  # above 0 so |x| = x+ + x-
boundaries = pulp.LpVariable.dicts('p', range(len(potential_triangles)), cat='Continuous')
cycles = pulp.LpVariable.dicts('q', range(len(potential_cycles)), cat='Continuous')

# objective
problem += pulp.lpDot(dim_1_indices.loc[potential_edges, 'filtration'].values, [cycle_pos[i] + cycle_neg[i] for i in potential_edges])

# constraints
for i in potential_edges:
    # can potentially add boudnaries
    bdry_expression = pulp.lpSum(boundary_matrix[i, j] * boundaries[j] for j in boundary_matrix[i, potential_triangles].indices)
    # can potentially add cycles
    cyc_expression = pulp.lpSum(cycle_basis[i, potential_cycles].tocsr()[j] * cycles[j] for j in cycle_basis[i, potential_cycles].tocsr().indices)
    # boundarys + cycles + initial cycle = new cycle
    problem += (bdry_expression + cyc_expression + init_cycle[i] == cycle_pos[i] - cycle_neg[i], f'row_{i}')

# solve it
problem.solve(lpsolver())

# collect the results
my_optimal_cycle = (
        dim_1_indices.loc[potential_edges]
            .assign(coefficient=[cycle_pos[i].value() - cycle_neg[i].value() for i in potential_edges])  # fill in the coefficient column
    )
my_optimal_cycle = my_optimal_cycle[~np.isclose(my_optimal_cycle['coefficient'], 0)].reset_index(drop=True)  # collect only nnz coeffiencnts

# make a plot
fig, ax = plt.subplots(figsize=(5, 5))
ax.axis('off')  # remove frame and lables
ax.axis('equal')  # equal scaleing

# plot the point
ax.scatter(pts[:, 0], pts[:, 1], c='k', s=10, zorder=2)  # points

# plot the initial cycle
lines = LineCollection(
            np.array((pts[initial_cycle['simplex'].str[0]], pts[initial_cycle['simplex'].str[1]])).swapaxes(0, 1),
            color='tab:orange',
            lw=1.5,
            zorder=3,
            label='Initial Cycle',
        )
ax.add_collection(lines)

# plot the optmial cycle
lines = LineCollection(
            np.array((pts[my_optimal_cycle['simplex'].str[0]], pts[my_optimal_cycle['simplex'].str[1]])).swapaxes(0, 1),
            color='tab:blue',
            lw=1.5,
            zorder=3,
            label='Optimal Cycle',
        )
ax.add_collection(lines)

# some final formatting
ax.legend(frameon=False, loc='upper left', fontsize='small')
fig.tight_layout()

The objective value for this cycle is equal to the one we get from OAT (with some numerical error).

In [ ]:
# check the objective
oat_objective = (oat_optimal_cycle['filtration'] * np.abs(oat_optimal_cycle['coefficient'])).sum()
my_objective = (my_optimal_cycle['filtration'] * np.abs(my_optimal_cycle['coefficient'])).sum()
assert np.isclose(oat_objective - my_objective, 0)

# plot the two
fig, ax = plt.subplots(figsize=(5, 5))
ax.axis('off')  # remove frame and lables
ax.axis('equal')  # equal scaleing

# plot the point
ax.scatter(pts[:, 0], pts[:, 1], c='k', s=10, zorder=2)  # points

# plot the initial cycle
lines = LineCollection(
            np.array((pts[oat_optimal_cycle['simplex'].str[0]], pts[oat_optimal_cycle['simplex'].str[1]])).swapaxes(0, 1),
            color='tab:orange',
            lw=1.5,
            zorder=3,
            label='Oat Optimal Cycle',
        )
ax.add_collection(lines)

# plot the optmial cycle
lines = LineCollection(
            np.array((pts[my_optimal_cycle['simplex'].str[0]], pts[my_optimal_cycle['simplex'].str[1]])).swapaxes(0, 1),
            color='tab:blue',
            ls=':',
            lw=1.5,
            zorder=3,
            label='My Optimal Cycle',
        )
ax.add_collection(lines)

# some final formatting
ax.legend(frameon=False, loc='upper left', fontsize='small')
fig.tight_layout()

# A Faster Way
The boundary matrix is very high rank, so we don't need to include every column, and doing so can add unneeded complexity to the problem by making it larger than it has to be. We could row reduce `boundary_matrix` to figure out which columns are important to include, but OAT does the work for us.

The `jordan_block_indices` from the factored object include the birth simplex of the minimal cycles and boundaries needed to create a basis for $A$ and $\partial_{n+1}$. From these, we can use `jordan_basis_vector` to get the cycle (or boundary) corresponding to that particular column in the basis. This is slower to build, but makes solving the LP faster, so is ideal if you plan to optimize more cycles.

In [ ]:
# get the indices
jordan_indices = factored.jordan_block_indices()
jordan_indices = jordan_indices[jordan_indices['dimension'] == 1].reset_index(drop=True)
jordan_indices['birth simplex'] = jordan_indices['birth simplex'].apply(tuple)  # make a hashable datatype

# loop, and build a basis
jordan_basis_data = pd.concat([
        cycle_vector_indices(factored.jordan_basis_vector(bs)).assign(col=i)  # each 'col' column is a unique feature
            for i, bs in enumerate(jordan_indices['birth simplex'])
    ])
jordan_basis = sparse.csr_array(
        (jordan_basis_data['coefficient'], (jordan_basis_data['row'], jordan_basis_data['col'])),
        shape=(len(dim_1_indices), len(jordan_indices)),  # has # of dim 1 simplices rows and # of dim 1 cycles columns
    )
del jordan_basis_data

# filter the indices to get important columns
jordan_indices = (  # this will be used to decide which columns to include later
        jordan_indices
            [['birth simplex', 'birth filtration', 'death filtration']]
            .rename(columns={'birth filtration': 'birth', 'death filtration': 'death'})
    )

# plot cycle basis (pixel where nonzero)
plt.spy(jordan_basis, marker=',', color='k')

**LP.** This Jordan Basis can be used in the LP in place of $A$ and $\partial_{n+1}$!

In [ ]:
# index sets
birth = homology.loc[idx, 'birth']
death = homology.loc[idx, 'death']
potential_edges = np.where(dim_1_indices['filtration'] <= birth)[0]  # curlR, allowed edges
jordan_mask = (jordan_indices[['birth', 'death']] <= np.array([birth, death])).all(axis=1)
potential_jordan_vectors = np.where(jordan_mask & (jordan_indices['birth simplex'] != tuple(birth_simplex)))[0]  # curlQ, allowed cycles
init_cycle = jordan_basis[potential_edges, np.where(jordan_indices['birth simplex'] == tuple(birth_simplex))[0][0]].toarray()  # xinit, initial cycle

# build the solver
problem = pulp.LpProblem('Jordan_Edge-Loss_Optimization', pulp.LpMinimize)

# variables we fit
cycle_pos = pulp.LpVariable.dicts('xpos', potential_edges, lowBound=0, cat='Continuous')  # cant do absolute value, instead x = x+ - x- and both are
cycle_neg = pulp.LpVariable.dicts('xneg', potential_edges, lowBound=0, cat='Continuous')  # above 0 so |x| = x+ + x-
jordan_cols = pulp.LpVariable.dicts('pq', range(len(potential_jordan_vectors)), cat='Continuous')

# objective
problem += pulp.lpDot(dim_1_indices.loc[potential_edges, 'filtration'].values, [cycle_pos[i] + cycle_neg[i] for i in potential_edges])

# constraints
for i in potential_edges:
    # can potentially add columns of the matrix
    jordan_basis_i = jordan_basis[i, potential_jordan_vectors].tocsr()
    jordan_expression = pulp.lpSum(jordan_basis_i[j] * jordan_cols[j] for j in jordan_basis_i.indices)
    # jordan columns + initial cycle = new cycle
    problem += (jordan_expression + init_cycle[i] == cycle_pos[i] - cycle_neg[i], f'row_{i}')

# solve it
problem.solve(lpsolver())

# collect the results
jordan_optimal_cycle = (
        dim_1_indices.loc[potential_edges]
            .assign(coefficient=[cycle_pos[i].value() - cycle_neg[i].value() for i in potential_edges])  # fill in the coefficient column
    )
jordan_optimal_cycle = jordan_optimal_cycle[~np.isclose(jordan_optimal_cycle['coefficient'], 0)].reset_index(drop=True)  # collect only nnz coeffiencnts

# make a plot
fig, ax = plt.subplots(figsize=(5, 5))
ax.axis('off')  # remove frame and lables
ax.axis('equal')  # equal scaleing

# plot the point
ax.scatter(pts[:, 0], pts[:, 1], c='k', s=10, zorder=2)  # points

# plot the initial cycle
lines = LineCollection(
            np.array((pts[initial_cycle['simplex'].str[0]], pts[initial_cycle['simplex'].str[1]])).swapaxes(0, 1),
            color='tab:orange',
            lw=1.5,
            zorder=3,
            label='Initial Cycle',
        )
ax.add_collection(lines)

# plot the optmial cycle
lines = LineCollection(
            np.array((pts[jordan_optimal_cycle['simplex'].str[0]], pts[jordan_optimal_cycle['simplex'].str[1]])).swapaxes(0, 1),
            color='tab:blue',
            lw=1.5,
            zorder=3,
            label='Optimal Cycle',
        )
ax.add_collection(lines)

# some final formatting
ax.legend(frameon=False, loc='upper left', fontsize='small')
fig.tight_layout()

And, it gets the same solution!

In [ ]:
# check the objective
my_objective = (my_optimal_cycle['filtration'] * np.abs(my_optimal_cycle['coefficient'])).sum()
jordan_objective = (jordan_optimal_cycle['filtration'] * np.abs(jordan_optimal_cycle['coefficient'])).sum()
assert np.isclose(jordan_objective - my_objective, 0)

# Bounding Chains

How do we find a bounding chain for the optimal cycle? Again, we turn to linear programming!

Remember, the bounding chain is the set of $(n+1)$-simplices (in this can triangles) whose boundary is the cycle. Therefore, we can solve for the bounding chain $\mathbf{x}^{BC}$ so that
$$
    \partial_{n+1} \mathbf{x}^{BC} = \mathbf{x}
$$
where $\mathbf{x}$ is the optimal cycle we found in the past step. Because $\partial_{n+1}$ is high rank, this problem has many solutions, many of which will overlap like we observed in the bounding chain we got from OAT. To avoid this, we'll solve for the *optimal bounding chain* by solving the linear program

\begin{align*}
    \underset{\mathbf{x}^{BC}}{\operatorname{minimize}} \quad & c^\top |\mathbf{x}^{BC}| \\
    \operatorname{subject~to} \quad & \partial_{n+1} \mathbf{x}^{BC} = \mathbf{x}
\end{align*}
where $c$ is a vector of triangle areas.

Note: Because we're including simplices at a higher filtration, this is a much larger problem and can take much longer. On my laptop this takes ~1 minute using the COIN solver. If you want it be faster get a Gurobi license, and it will take seconds.

In [ ]:
# find the area of each triangle, use Heron's formula https://en.wikipedia.org/wiki/Heron's_formula
a = pd.DataFrame(dim_2_indices['simplex'].str[:2]).merge(dim_1_indices, on='simplex', how='left')['filtration']
b = pd.DataFrame(dim_2_indices['simplex'].str[:1] + dim_2_indices['simplex'].str[2:]).merge(dim_1_indices, on='simplex', how='left')['filtration']
c = pd.DataFrame(dim_2_indices['simplex'].str[1:]).merge(dim_1_indices, on='simplex', how='left')['filtration']
s = (a + b + c) / 2
area = np.sqrt(s * (s - a) * (s - b) * (s - c)).values
del a, b, c, s

# the bounding chian exists at the death time, filter to then
death = homology.loc[idx, 'death']
potential_edges = np.where(dim_1_indices['filtration'] <= death)[0]  # curlR, allowed edges
potential_triangles = np.where(dim_2_indices['filtration'] <= death)[0]  # curlP, allowed triangles

# get a vector representation of the optimal cycles
cycle_indices = cycle_vector_indices(jordan_optimal_cycle)
cycle_vector = np.zeros(potential_edges[-1] + 1)
cycle_vector[cycle_indices['row']] = cycle_indices['coefficient']

# build the solver
problem = pulp.LpProblem('Minimal_Bounding_Chain', pulp.LpMinimize)

# variables we fit
bounding_chain_pos = pulp.LpVariable.dicts('xbcpos', potential_triangles, lowBound=0, cat='Continuous')  # cant do absolute value, instead x = x+ - x- and both are
bounding_chain_neg = pulp.LpVariable.dicts('xbcneg', potential_triangles, lowBound=0, cat='Continuous')  # above 0 so |x| = x+ + x-

# objective
# add a little bit to the area so there is a unique solution
# there are many options for triangles that would fit within the region, this picks one with fewer tirnalges
problem += pulp.lpDot(area + 1e-3, [bounding_chain_pos[i] + bounding_chain_neg[i] for i in potential_triangles])

# constraints
for i in potential_edges:
    # can potentially add columns of the matrix
    boundary_expression = pulp.lpSum(boundary_matrix[i, j] * (bounding_chain_pos[j] - bounding_chain_neg[j]) for j in boundary_matrix[i, potential_triangles].indices)
    # jordan columns + initial cycle = new cycle
    problem += (boundary_expression == cycle_vector[i], f'row_{i}')

# solve it
problem.solve(lpsolver())

# collect the results
optimal_bounding_chain = (
        dim_2_indices.loc[potential_triangles]
            .assign(coefficient=[bounding_chain_pos[i].value() - bounding_chain_neg[i].value() for i in potential_triangles])  # fill in the coefficient column
    )
optimal_bounding_chain = optimal_bounding_chain[optimal_bounding_chain['coefficient'] != 0].reset_index(drop=True)  # collect only nnz coeffiencnts

# make a plot
fig, ax = plt.subplots(figsize=(5, 5))
ax.axis('off')  # remove frame and lables
ax.axis('equal')  # equal scaleing

# plot the point
ax.scatter(pts[:, 0], pts[:, 1], c='k', s=10, zorder=2)  # points

# plot the cycle
lines = LineCollection(  # we could loop and plot each line individually, but that takes longer to render. linecollections will be fast
            np.array((pts[jordan_optimal_cycle['simplex'].str[0]], pts[jordan_optimal_cycle['simplex'].str[1]])).swapaxes(0, 1),  # points along each ot the lines
            color='k',
            lw=1.5,
            zorder=3
        )
ax.add_collection(lines)

# plot the bounding chain
tris = PatchCollection(  # again, we could loop and use fill but thats slower than a PatchCollection
        [Polygon(pts[list(s)]) for s in optimal_bounding_chain['simplex']],
        color='gray',
        edgecolor=None,  # no edges
        alpha=0.25,
        zorder=1
    )
ax.add_collection(tris)

# some final formatting
fig.tight_layout()

# Triangle-Loss Optimization
So far, we've solved for the smallest cycle boundary. This, however, is a choice we made when setting up the problem, and there are alternatives.

One alternative is *triangle-loss optimization*. Instead of searching for the smallest cycle, we'll search to the cycle that bounds the smallest area (i.e. the cycle that has the smallest bounding chain). This will in general create a cycle that's slightly longer, but has a smaller area within it.

To find the triangle-loss cycle representative, we solve

\begin{align*}
    \underset{\mathbf{x}, \mathbf{x}^{BC} \mathbf{p}, \mathbf{q}}{\operatorname{minimize}} \quad & c^\top | \mathbf{x}^{BC} | \\
    \operatorname{subject~to} \quad & \mathbf{x} = \partial_{n+1} \mathbf{x}^{BC} \\
    & \mathbf{x}[\mathcal{R}] = \mathbf{x}^{Init}[\mathcal{R}] + \partial_{n+1} [\mathcal{R}, \mathcal{P}] \mathbf{p} + A [\mathcal{R}, \mathcal{q}] \mathbf{q}.
\end{align*}

In [ ]:
# index sets
birth = homology.loc[idx, 'birth']
death = homology.loc[idx, 'death']
potential_edges = np.where(dim_1_indices['filtration'] <= birth)[0]  # curlR, allowed edges
jordan_mask = (jordan_indices[['birth', 'death']] <= np.array([birth, death])).all(axis=1)
potential_jordan_vectors = np.where(jordan_mask & (jordan_indices['birth simplex'] != tuple(birth_simplex)))[0]  # curlQ, allowed cycles
init_cycle = jordan_basis[potential_edges, np.where(jordan_indices['birth simplex'] == tuple(birth_simplex))[0][0]].toarray()  # xinit, initial cycle
potential_bc_edges = np.where(dim_1_indices['filtration'] <= death)[0]  # curlR, allowed edges
potential_bc_triangles = np.where(dim_2_indices['filtration'] <= death)[0]  # curlP, allowed triangles

# build the solver
problem = pulp.LpProblem('Triangle-Loss_Optimization', pulp.LpMinimize)

# variables we fit
bounding_chain_pos = pulp.LpVariable.dicts('xbcpos', potential_triangles, lowBound=0, cat='Continuous')  # cant do absolute value, instead x = x+ - x- and both are
bounding_chain_neg = pulp.LpVariable.dicts('xbcneg', potential_triangles, lowBound=0, cat='Continuous')  # above 0 so |x| = x+ + x-
cycle_coef = pulp.LpVariable.dicts('xpos', potential_edges, cat='Continuous')
jordan_cols = pulp.LpVariable.dicts('pq', range(len(potential_jordan_vectors)), cat='Continuous')

# objective
problem += pulp.lpDot(area + 1e-3, [bounding_chain_pos[i] + bounding_chain_neg[i] for i in potential_triangles])

# constraint 1: cycle
for i in potential_edges:
    # can potentially add columns of the matrix
    jordan_basis_i = jordan_basis[i, potential_jordan_vectors].tocsr()
    jordan_expression = pulp.lpSum(jordan_basis_i[j] * jordan_cols[j] for j in jordan_basis_i.indices)
    # jordan columns + initial cycle = new cycle
    problem += (jordan_expression + init_cycle[i] == cycle_coef[i], f'row_{i}')

# constraint 2: bounding chain
for i in potential_bc_edges:
    # can potentially add columns of the matrix
    boundary_expression = pulp.lpSum(boundary_matrix[i, j] * (bounding_chain_pos[j] - bounding_chain_neg[j]) for j in boundary_matrix[i, potential_triangles].indices)
    # jordan columns + initial cycle = new cycle
    problem += (boundary_expression == cycle_coef.get(i, 0), f'bc_row_{i}')

# solve it
problem.solve(lpsolver())

# collect the results
tri_loss_optimal_cycle = (
        dim_1_indices.loc[potential_edges]
            .assign(coefficient=[cycle_coef[i].value() for i in potential_edges])  # fill in the coefficient column
    )
tri_loss_optimal_cycle = tri_loss_optimal_cycle[~np.isclose(tri_loss_optimal_cycle['coefficient'], 0)].reset_index(drop=True)  # collect only nnz coeffiencnts
tri_loss_optimal_bounding_chain = (
        dim_2_indices.loc[potential_bc_triangles]
            .assign(coefficient=[bounding_chain_pos[i].value() - bounding_chain_neg[i].value() for i in potential_bc_triangles])  # fill in the coefficient column
    )
tri_loss_optimal_bounding_chain = tri_loss_optimal_bounding_chain[~np.isclose(tri_loss_optimal_bounding_chain['coefficient'], 0)].reset_index(drop=True)  # collect only nnz coeffiencnts

# make a plot
fig, ax = plt.subplots(figsize=(5, 5))
ax.axis('off')  # remove frame and lables
ax.axis('equal')  # equal scaleing

# plot the point
ax.scatter(pts[:, 0], pts[:, 1], c='k', s=10, zorder=2)  # points

# plot the initial cycle
lines = LineCollection(
            np.array((pts[jordan_optimal_cycle['simplex'].str[0]], pts[jordan_optimal_cycle['simplex'].str[1]])).swapaxes(0, 1),
            color='tab:orange',
            lw=1.5,
            zorder=3,
            label='Edge-Loss Cycle',
        )
ax.add_collection(lines)

# plot the optmial cycle
lines = LineCollection(
            np.array((pts[tri_loss_optimal_cycle['simplex'].str[0]], pts[tri_loss_optimal_cycle['simplex'].str[1]])).swapaxes(0, 1),
            color='tab:blue',
            lw=1.5,
            zorder=3,
            label='Triangle-Loss Cycle',
        )
ax.add_collection(lines)

# some final formatting
ax.legend(frameon=False, loc='upper left', fontsize='small')
fig.tight_layout()

What we see is that the edge-loss cycle smoothly surrounds the hole, but can be farther from the actual empty space in the point set at times. The triangle-loss, in contrast, cycle is more jagged, but more closely surrounds the hole.

# Next Steps
If you're done, try to run this code on cycles in other point-sets. You can do that easily by replacing `pts` at the very top with any point set of your choosing. Try to look at:
1. Optimizing multiple cycles in a point set. You can choose which cycle is optimized using the `idx` variable. Do the optimized cycles each surround the correct hole?
2. Optimizing the hole in an annulus with a filled region in the middle. Try both edge- and triangle-loss optimization. Are they different? How?

Note: If the number of points grows substantially, these can start to take much longer to solve since the number of triangles grows a lot. For now, try to stick to point sets with a maximum of a couple-hundred points. If you need to scale up, look into getting a license for third-party LP solvers. Gurobi and CPLEX are both much faster and free for academic use, Gurobi also has a python library gurobipy which can initialize problems much faster than PuLP.

In [ ]:
# three annuluses, use max_filtration = 1.5
n = 50  # number of points per annululs
r = 0.75  # innner radius
R = 1  # outer radius
np.random.seed(1)
annus = []
for _ in range(3):
    rad = np.sqrt(np.random.uniform(r**2, R**2, n))  # radius of each point, evenly distributed
    theta = np.random.uniform(0, 2 * np.pi, n)  # angle of each point
    x = rad * np.cos(theta)  # x vaues
    y = rad * np.sin(theta)  # y values
    annus.append(np.array((x, y)).T)  # concatenated df
pts = np.vstack((
        annus[0] + 2 / np.sqrt(3) * R * np.array([0, 1]),
        annus[1] + 2 / np.sqrt(3) * R * np.array([0.5 * np.sqrt(3), -0.5]),
        annus[2] + 2 / np.sqrt(3) * R * np.array([-0.5 * np.sqrt(3), -0.5]),
    ))

# plot it
fig, ax = plt.subplots()
fig.set_figheight(3)
fig.set_figwidth(3)
ax.axis('equal')
ax.set_axis_off()
ax.plot(pts[:, 0], pts[:, 1], 'k.', ms=2)
fig.tight_layout()

In [ ]:
# annulus with an inner area, use max_filtration = 0.75
n_in = 50  # number of points in the interior
n_out = 100  # number of points in the expteriot
circ_rad = 0.5  # interetior circle radius
inner_rad = 1  # innner annulus radius
outer_rad = 1.25  # outer radius
np.random.seed(1)
r = np.sqrt(np.hstack((np.random.uniform(0, circ_rad**2, n_in), np.random.uniform(inner_rad**2, outer_rad**2, n_out))))
theta = np.random.uniform(0, 2 * np.pi, n_in + n_out)
pts = np.array((
        r * np.cos(theta),  # x vales
        r * np.sin(theta),  # y vales
    )).T

# plot it
fig, ax = plt.subplots()
fig.set_figheight(3)
fig.set_figwidth(3)
ax.axis('equal')
ax.set_axis_off()
ax.plot(pts[:, 0], pts[:, 1], 'k.', ms=2)
fig.tight_layout()

# Some Final Notes
Cycle optimization is an incredibly powerful technique that can help to derive meaningful insights from peristant homology. In theory, these techniques can be applied to any dimension cycle you want, but there are computational challenges for higher dimensions since the number of $n$-simplicies increases substantially for higher $n$ (To check, compare `len(dim_1_indices)` to `len(dim_2_indices)`).

For more information on the topic, there's quite a bit of liturature to check out. For more thourough treatment of the topics in this notebook, I'd suggest:
- Escolar, Emerson G., and Yasuaki Hiraoka. "Computing optimal cycles of homology groups." A Mathematical Approach to Research Problems of Science and Technology: Theoretical Basis and Developments in Mathematical Modeling (2014): 101-118.
- Obayashi, Ippei. "Volume-optimal cycle: Tightest representative cycle of a generator in persistent homology." SIAM Journal on Applied Algebra and Geometry 2.4 (2018): 508-534.
- Li, Lu, et al. "Minimal cycle representatives in persistent homology using linear programming: An empirical study with user’s guide." Frontiers in artificial intelligence 4 (2021): 681117.